#### 2.b) Write an `R` function to calculate the log-likelihood function from 2a above.  Your function should take 3 arguments: (1) a vector $\beta$ of all (four) parameters, (2) an $n \times k$ matrix $X$, and (3) a vector or $n \times 1$ matrix $y$.


In [3]:
tb <- read.csv("C:/Users/secar/OneDrive/Documents/UCLA/Econometrics/trading_behavior.csv")

loglik_pois <- function(beta, X, y) {
  y <- as.vector(y)
  
  eta <- as.vector(X %*% beta)
  
  logmat <- sum( y * eta - exp(eta) - lgamma(y + 1) )
  return(logmat)
}

#### 2.c) Use `optim()` and your function from 1b to find $\hat{\beta}_\text{MLE}$ as well as their standard errors.

In [ ]:
neg_loglik_pois <- function(beta, X, y) {
  -loglik_pois(beta, X, y)
}

y <- tb$numtrades
X <- model.matrix(~ program + finlittest, data = tb)  
beta_init <- rep(0, ncol(X))

fit_poisson <- optim(par = beta_init,
                     fn  = neg_loglik_pois,
                     X   = X,
                     y   = y,
                     hessian = TRUE)

#### 2.d) Fit the model using `glm()`.  Compare your results to the output from `optim` in question 2c above.

In [5]:
pois_glm <- glm(numtrades ~ program + finlittest,
                family = poisson(link = "log"),
                data   = tb)

summary(pois_glm)

fit_poisson <- optim(par    = beta_init,
                     fn     = neg_loglik_pois,
                     X      = X,
                     y      = y,
                     hessian = TRUE)

beta_hat_optim <- fit_poisson$par
beta_hat_optim

vcov_optim <- solve(fit_poisson$hessian)       
se_optim   <- sqrt(diag(vcov_optim))
se_optim

cbind(glm_coef = coef(pois_glm),
      optim_coef = beta_hat_optim)

cbind(glm_se = summary(pois_glm)$coef[, "Std. Error"],
      optim_se = se_optim)


Call:
glm(formula = numtrades ~ program + finlittest, family = poisson(link = "log"), 
    data = tb)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.91708    0.92579  -6.391 1.64e-10 ***
programMFE  -0.71405    0.32001  -2.231  0.02566 *  
programMSBA -1.08386    0.35825  -3.025  0.00248 ** 
finlittest   0.07015    0.01060   6.619 3.63e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 287.67  on 199  degrees of freedom
Residual deviance: 189.45  on 196  degrees of freedom
AIC: 373.5

Number of Fisher Scoring iterations: 6


[1] -5.91714462 -0.71348934 -1.08372911  0.07015372

[1] 0.92365546 0.31992292 0.35822540 0.01056082

,glm_coef,optim_coef
(Intercept),-5.9170752,-5.91714462
programMFE,-0.7140499,-0.71348934
programMSBA,-1.0838591,-1.08372911
finlittest,0.0701524,0.07015372


,glm_se,optim_se
(Intercept),0.9257925,0.92365546
programMFE,0.3200149,0.31992292
programMSBA,0.3582530,0.35822540
finlittest,0.0105992,0.01056082


The coefficients from glm() and optim() are almost identical, up to small numerical differences- showing that optim successfully found the Poisson MLE. The standard errors from glm() match those computed from the inverse Hessian pretty closely, with slight differences being a matter of rounding error. This confirms that both approaches are computing the same Poisson MLE; glm() is just a convenient wrapper that automates the optimization and variance calculation.

#### where $\ell_n(\cdot)$ is the log likelihood function, $\hat{\theta}$ is the MLE, and $\tilde{\theta}$ is a constrained parameter vector (e.g., suppose a Null Hypothesis is that $\theta_2=0$ & $\theta_3=0$).  The Likelihood Ratio test statistic $LR_n$ has an asymptotic chi-squared distribution with $k$ degrees of freedom (i.e., $\chi^2_k$ where $k$ is the number of constraints imposed under the Null Hypothesis).

#### Test the joint hypothesis that $\beta_2=0$ & $\beta_3=0$ at the 95% confidence level using a Likelihood Ratio test. Specifically, use your log-likelihood function from 2b above and your parameter estimates from 2c above to calculate $\ell_n(\hat{\beta}_\text{MLE})$.  Then replace $\beta_2$ and $\beta_3$ with their hypothesized values and re-calculate the log-likehood (ie, $\ell_n([\hat{\beta}_1,0,0,\hat{\beta}_4])$.  Next, compute $LR_n$ and compare the value to the cut-off of a chi-squared distribution with 2 degrees of freedom to assess whether or not you reject the Null Hypothesis.

In [6]:
beta_hat <- fit_poisson$par

nll_hat <- neg_loglik_pois(beta_hat, X, y)

beta_tilde <- beta_hat
beta_tilde[2] <- 0
beta_tilde[3] <- 0

nll_tilde <- neg_loglik_pois(beta_tilde, X, y)

LR_n <- 2 * (nll_tilde - nll_hat)   

crit <- qchisq(0.95, df = 2)
pval <- 1 - pchisq(LR_n, df = 2)

LR_n
crit
pval

[1] 23.57845

[1] 5.991465

[1] 7.585875e-06

The LRn value 23.58 is substantially larger than the 5% chi-square critical value of 5.99 with 2 degrees of freedom. The associated p-value is 7.6x10^-6, which is way below .05. We can confidently reject the joint null hypothesis that beta_tilde[2] and beta_tilde[3] are equal to 0. The finlittest jointly improves the fit of the Poisson regression model for number of trades.